In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/classification-competition-cryptonite/sample_submission.csv
/kaggle/input/classification-competition-cryptonite/train.csv
/kaggle/input/classification-competition-cryptonite/test.csv


In [2]:
X_full = pd.read_csv('/kaggle/input/classification-competition-cryptonite/train.csv')
X_test_full = pd.read_csv('/kaggle/input/classification-competition-cryptonite/test.csv')
X_full.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,1084.0,D-penicillamine,17664.0,F,N,N,N,N,0.5,273.0,3.65,23.0,648.0,52.70,59.0,316.0,10.0,2.0,C
1,1,790.0,D-penicillamine,23376.0,F,N,Y,N,N,8.4,NaN,3.15,161.0,1790.0,181.35,NaN,298.0,13.6,4.0,D
2,2,1367.0,NaN,13149.0,F,NaN,NaN,NaN,N,0.6,NaN,3.03,NaN,NaN,NaN,NaN,402.0,11.4,1.0,C
3,3,2721.0,Placebo,17809.0,F,N,Y,N,N,0.9,NaN,3.80,18.0,1868.0,97.65,NaN,327.0,9.5,2.0,C
4,4,989.0,NaN,12369.0,F,NaN,NaN,NaN,N,1.8,NaN,3.58,NaN,NaN,NaN,NaN,244.0,10.4,3.0,C


In [3]:
from sklearn.model_selection import train_test_split
y = X_full['Status']
X_full.drop(['Status'], axis = 1, inplace = True)

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, train_size = 0.8, test_size = 0.2, random_state = 0)

categorical_cols = [col for col in X_train_full.columns if X_train_full[col].nunique() < 10 and X_train_full[col].dtype == 'object']
numerical_cols = [col for col in X_train_full.columns if X_train_full[col].dtype in ['int64','float64']]

my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

X_train.head()


,Drug,Sex,Ascites,Hepatomegaly,Spiders,Edema,id,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
13136,D-penicillamine,F,N,Y,N,N,13136,708.0,22646.0,6.1,328.0,3.60,58.0,1440.0,153.45,134.0,156.0,10.2,4.0
3536,NaN,F,NaN,NaN,NaN,N,3536,1067.0,25006.0,1.4,NaN,3.58,NaN,NaN,NaN,NaN,141.0,10.0,4.0
10296,D-penicillamine,M,N,Y,N,N,10296,1084.0,18137.0,7.1,426.0,3.61,217.0,2374.0,170.50,218.0,344.0,10.6,3.0
2137,Placebo,F,N,Y,Y,S,2137,1216.0,16050.0,1.2,432.0,3.53,123.0,975.0,125.55,155.0,295.0,11.5,4.0
14775,Placebo,F,N,N,N,N,14775,4190.0,13514.0,0.6,NaN,3.50,50.0,843.0,55.80,NaN,146.0,10.3,2.0


In [4]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

numerical_transformer = SimpleImputer(strategy = 'most_frequent')
categorical_transformer = Pipeline(steps = [('imputer', SimpleImputer(strategy = 'most_frequent')),('onehotencoder',OneHotEncoder(handle_unknown = 'ignore'))])

preprocessor = ColumnTransformer(transformers = [('numerical', numerical_transformer, numerical_cols),('categorical', categorical_transformer, categorical_cols)])

model_1 = LogisticRegression(multi_class = 'multinomial', solver = 'lbfgs')

model = Pipeline(steps = [('preprocess', preprocessor),('model', model_1)])

model.fit(X_train,y_train)
pred = model.predict(X_valid)
accuracy = accuracy_score(y_valid, pred)
print(accuracy)

0.778


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [5]:
probability = model.predict_proba(X_test)

output = pd.DataFrame(probability, columns = ['Status_C', 'Status_CL', 'Status_D'])
output.insert(0, 'id', X_test['id'])
print(output)

output.to_csv('submission.csv', index = False)

         id  Status_C  Status_CL  Status_D
0     15000  0.694518   0.011122  0.294361
1     15001  0.082686   0.007620  0.909694
2     15002  0.375042   0.165236  0.459722
3     15003  0.590293   0.010017  0.399690
4     15004  0.814853   0.008060  0.177088
...     ...       ...        ...       ...
9995  24995  0.928725   0.009558  0.061716
9996  24996  0.835042   0.009771  0.155187
9997  24997  0.817003   0.017237  0.165761
9998  24998  0.769253   0.072876  0.157871
9999  24999  0.469808   0.004633  0.525559

[10000 rows x 4 columns]
